# Billiard project

## 1) Square case

Assume that there is no friction and the Descartes rules are satisfied.

A game to play with this billiard would be to create words associated to trajectories:

We start with an empty word. 
Then, when the ball hit the top or bottom part we add an H (H for horizontal) to the word and when the ball hit the left or right part, we add a V (V for vertical).
Once words are created, we are going to analyse them with respect to the angle use to start the trajectory. We will restrict to words of size < 2000


Let's start by creating a billard table.

In [3]:
import numpy as np  
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as pch
from tkinter import *
%matplotlib notebook
import tkinter
import time
from math import *


In [4]:
from tkinter import *
#We define the function of the ball's movement: 
mot = str()
def deplacement():
    global dx, dy
    if canvas.coords(balle1)[3]>400:
        dy=-1*dy
    if (canvas.coords(balle1)[2]>400):
        dx=-1*dx
    if (canvas.coords(balle1)[1]<0):
        dy=-1*dy
    if (canvas.coords(balle1)[0]<0):
        dx=-1*dx
            
    #Ball's movement :
    canvas.move(balle1,dx,dy)
    #We reiterate this function 
    tk.after(20,deplacement)
 
#Deplacement de la balle au départ:
dx=3
dy=5


#creation du mot 
nom='mot'
index=0
while index < len(nom):
    if canvas.coords(balle1) [3]>400:
        print(mot[index] + 'h')
    if (canvas.coords(balle1)[2]>400):
        print(mot[index] + 'v')
        
    if (canvas.coords(balle1)[1]<0):
        print(mot[index]+ 'h')
    
    if (canvas.coords(balle1)[0]<0):
        print(mot[index] + 'v')
    
 
# We create a new window with the tkinter function 
window = tkinter.Tk()
tk = Tk()
tk.title("Square Billiard Game")
label_title=Label(tk,text="Welcome to billiard game")
label_title.pack()
canvas = tkinter.Canvas(tk, width="400", height="400",background='#096a09') #Dimension square are 400x400
canvas.pack(padx=10,pady=10)
 
#We create an "exit" button
Bouton_Quitter=Button(tk, text ='Quitter', command = tk.destroy)
#On ajoute l'affichage du bouton dans la fenêtre tk:
Bouton_Quitter.pack()
 
#We create a ball 
balle1 = canvas.create_oval(10,10,30,30,fill='white')
 
deplacement()
 
#On lance la boucle principale:
tk.mainloop()





NameError: name 'canvas' is not defined

In [9]:
window.geometry("600x400+600+300")
canvas=Canvas(window, width="400",height="400",background='#096A09')

bool = True    
while 1:
    if(x == 350):
        a = False
    if(x==0):
        a=True
    if(a):
        canvas.move(rond,10,0)
        x += 10
    else:
        canvas.move(rond,-10,0)
        x+= -10
        
    time.sleep(0.1)
    canvas.update()

TclError: invalid command name ".!canvas"

In [1]:
nom = 'mot'
index = 0
while index < len(nom):
    if 
    print( + ' *'), 
    index = index + 1

J *
a *
c *
q *
u *
e *
l *
i *
n *
e *
